# மாதிரி 02: OpenAI SDK ஒருங்கிணைப்பு

இந்த நோட்புக் OpenAI Python SDK உடன் மேம்பட்ட ஒருங்கிணைப்பை விளக்குகிறது, Microsoft Foundry Local மற்றும் Azure OpenAI இரண்டையும் ஆதரிக்கிறது, ஸ்ட்ரீமிங் பதில்கள் மற்றும் சரியான பிழை கையாளுதலுடன்.

## மேலோட்டம்

இந்த மாதிரி காட்டுகிறது:
- Foundry Local மற்றும் Azure OpenAI இடையே எளிதான மாறுதல்
- சிறந்த பயனர் அனுபவத்திற்காக ஸ்ட்ரீமிங் உரையாடல் முடிவுகள்
- FoundryLocalManager SDK-ஐ சரியாகப் பயன்படுத்துதல்
- வலுவான பிழை கையாளுதல் மற்றும் மாற்று வழிகள்
- உற்பத்திக்கு தயாரான குறியீட்டு முறைமைகள்


## முன்பதிவுகள்

- **Foundry Local**: நிறுவப்பட்டு இயங்கும் நிலையில் (உள்ளூர் முன்னறிவிப்பு பயன்பாட்டிற்காக)
- **Python**: 3.8 அல்லது அதற்கு மேற்பட்டது OpenAI SDK உடன்
- **Azure OpenAI**: செல்லுபடியாகும் முடிவுநிலை மற்றும் API விசை (மேக முன்னறிவிப்பு பயன்பாட்டிற்காக)

### சார்புகளை நிறுவவும்


In [ ]:
# Install required packages
!pip install openai foundry-local-sdk

## நூலகங்களை இறக்குமதி செய்து அமைக்கவும்


In [ ]:
import os
import sys
from openai import OpenAI
import time
from typing import Tuple

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, manual configuration will be used")

## கட்டமைப்பு விருப்பங்கள்

சரியான சூழல் மாறிகளை அமைப்பதன் மூலம் Azure OpenAI (மேகம்) அல்லது Foundry Local (சாதனத்தில்) இடையே தேர்வு செய்யவும்.


### விருப்பம் 1: Azure OpenAI அமைப்பு

உங்கள் Azure OpenAI சான்றுகளை Uncomment செய்து அமைக்கவும்:


In [ ]:
# Azure OpenAI Configuration
# Uncomment and set your actual values

# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key-here"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"  # e.g., "gpt-4"

print("Azure OpenAI configuration ready (if credentials are set)")

### விருப்பம் 2: Foundry உள்ளூர் கட்டமைப்பு

உள்ளூர் முடிவெடுப்புக்கான இயல்புநிலை அமைப்புகள்:


In [ ]:
# Foundry Local Configuration (default)
FOUNDRY_MODEL = "phi-4-mini"  # Change to your preferred model
FOUNDRY_BASE_URL = "http://localhost:51211"
FOUNDRY_API_KEY = ""  # Usually empty for local

print(f"Foundry Local configuration ready with model: {FOUNDRY_MODEL}")

## கிளையன்ட் தொழிற்சாலை செயல்பாடுகள்

இந்த செயல்பாடுகள் உங்கள் கட்டமைப்பின் அடிப்படையில் சரியான OpenAI கிளையன்டை உருவாக்குகின்றன:


In [ ]:
def create_azure_client() -> Tuple[OpenAI, str]:
    """Create Azure OpenAI client."""
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if not azure_endpoint or not azure_api_key:
        raise ValueError("Azure OpenAI endpoint and API key are required")
    
    model = os.environ.get("MODEL", "your-deployment-name")
    client = OpenAI(
        base_url=f"{azure_endpoint}/openai",
        api_key=azure_api_key,
        default_query={"api-version": azure_api_version},
    )
    
    print(f"🌐 Azure OpenAI client created with model: {model}")
    return client, model


def create_foundry_client() -> Tuple[OpenAI, str]:
    """Create Foundry Local client with SDK management."""
    alias = FOUNDRY_MODEL
    
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            print(f"🔄 Initializing Foundry Local with model: {alias}...")
            manager = FoundryLocalManager(alias)
            model_info = manager.get_model_info(alias)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            
            print(f"✅ Foundry Local SDK initialized")
            print(f"   Endpoint: {manager.endpoint}")
            print(f"   Model: {model_info.id}")
            return client, model_info.id
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{FOUNDRY_BASE_URL}/v1",
        api_key=FOUNDRY_API_KEY
    )
    
    print(f"🔧 Manual Foundry Local configuration")
    print(f"   Endpoint: {FOUNDRY_BASE_URL}/v1")
    print(f"   Model: {alias}")
    return client, alias

## கிளையன்டை தொடங்கவும்

இது Azure OpenAI அல்லது Foundry Local ஐ பயன்படுத்த வேண்டுமா என்பதை தானாகவே கண்டறிகிறது:


In [ ]:
def initialize_client() -> Tuple[OpenAI, str, str]:
    """Initialize the appropriate OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    
    if azure_endpoint and azure_api_key:
        print("🌐 Azure OpenAI configuration detected")
        try:
            client, model = create_azure_client()
            return client, model, "azure"
        except Exception as e:
            print(f"❌ Azure OpenAI initialization failed: {e}")
            print("🔄 Falling back to Foundry Local...")
    
    # Use Foundry Local
    print("🏠 Using Foundry Local configuration")
    try:
        client, model = create_foundry_client()
        return client, model, "foundry"
    except Exception as e:
        print(f"❌ Foundry Local initialization failed: {e}")
        raise

# Initialize the client
print("Initializing OpenAI client...")
print("=" * 50)
client, model, provider = initialize_client()
print("=" * 50)
print(f"✅ Initialization complete! Using {provider} with model: {model}")

## அடிப்படை உரையாடல் நிறைவு

ஒரு எளிய உரையாடல் நிறைவை சோதிக்கவும்:


In [ ]:
def simple_chat(prompt: str, max_tokens: int = 150) -> str:
    """Send a simple chat message and get response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test basic chat
test_prompt = "Say hello from the SDK quickstart and explain what you are in one sentence."

print(f"👤 User: {test_prompt}")
print("\n🤖 Assistant:")
response = simple_chat(test_prompt)
print(response)

## ஸ்ட்ரீமிங் சொடுக்கல் நிறைவு

சிறந்த பயனர் அனுபவத்திற்காக ஸ்ட்ரீமிங் பதில்களை காட்டுங்கள்:


In [ ]:
def streaming_chat(prompt: str, max_tokens: int = 300) -> str:
    """Send a chat message with streaming response."""
    try:
        print("🤖 Assistant (streaming):")
        
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        error_msg = f"Error: {e}"
        print(error_msg)
        return error_msg

# Test streaming chat
streaming_prompt = "Explain the key benefits of using Microsoft Foundry Local for AI development. Include aspects like privacy, performance, and cost."

print(f"👤 User: {streaming_prompt}\n")
streaming_response = streaming_chat(streaming_prompt)

## பல முறை உரையாடல்

உரையாடல் சூழலை பராமரிப்பதை விளக்கவும்:


In [ ]:
class ConversationManager:
    """Manages multi-turn conversations with context."""
    
    def __init__(self, system_prompt: str = None):
        self.messages = []
        if system_prompt:
            self.messages.append({"role": "system", "content": system_prompt})
    
    def send_message(self, user_message: str, max_tokens: int = 200) -> str:
        """Send a message and get response while maintaining context."""
        # Add user message to conversation
        self.messages.append({"role": "user", "content": user_message})
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=self.messages,
                max_tokens=max_tokens
            )
            
            assistant_message = response.choices[0].message.content
            
            # Add assistant response to conversation
            self.messages.append({"role": "assistant", "content": assistant_message})
            
            return assistant_message
        except Exception as e:
            return f"Error: {e}"
    
    def get_conversation_length(self) -> int:
        """Get the number of messages in the conversation."""
        return len(self.messages)

# Create conversation manager with system prompt
system_prompt = "You are a helpful AI assistant specialized in explaining AI and machine learning concepts. Be concise but informative."
conversation = ConversationManager(system_prompt)

# Multi-turn conversation example
conversation_turns = [
    "What is the difference between AI inference on-device vs in the cloud?",
    "Which approach is better for privacy?",
    "What about performance and latency considerations?"
]

for i, turn in enumerate(conversation_turns, 1):
    print(f"\n{'='*60}")
    print(f"Turn {i}")
    print(f"{'='*60}")
    print(f"👤 User: {turn}")
    
    response = conversation.send_message(turn)
    print(f"\n🤖 Assistant: {response}")

print(f"\n📊 Conversation summary: {conversation.get_conversation_length()} messages total")

## செயல்திறன் ஒப்பீடு

விதவிதமான சூழல்களில் பதில் நேரங்களை ஒப்பிடுங்கள்:


In [ ]:
def benchmark_response_time(prompt: str, iterations: int = 3) -> dict:
    """Benchmark response time for a given prompt."""
    times = []
    responses = []
    
    for i in range(iterations):
        start_time = time.time()
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=50  # Keep responses short for timing
            )
            
            end_time = time.time()
            response_time = end_time - start_time
            
            times.append(response_time)
            responses.append(response.choices[0].message.content)
            
        except Exception as e:
            print(f"Error in iteration {i+1}: {e}")
    
    if times:
        avg_time = sum(times) / len(times)
        min_time = min(times)
        max_time = max(times)
        
        return {
            "average_time": avg_time,
            "min_time": min_time,
            "max_time": max_time,
            "all_times": times,
            "sample_response": responses[0] if responses else None
        }
    
    return {"error": "No successful responses"}

# Benchmark different types of prompts
benchmark_prompts = [
    "What is AI?",
    "Explain machine learning in simple terms.",
    "List 3 benefits of edge computing."
]

print(f"⏱️  Performance Benchmark ({provider} - {model})")
print("=" * 60)

for prompt in benchmark_prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    results = benchmark_response_time(prompt)
    
    if "error" not in results:
        print(f"   ⏰ Average time: {results['average_time']:.2f}s")
        print(f"   ⚡ Fastest: {results['min_time']:.2f}s")
        print(f"   🐌 Slowest: {results['max_time']:.2f}s")
        print(f"   📄 Sample response: {results['sample_response'][:100]}...")
    else:
        print(f"   ❌ {results['error']}")

## மேம்பட்ட கட்டமைப்பு மற்றும் பிழை கையாளுதல்

பல்வேறு அளவுருக்கள் மற்றும் பிழை நிலைகளை சோதிக்கவும்:


In [ ]:
def test_different_parameters():
    """Test chat completions with different parameters."""
    prompt = "Write a creative short story about AI."
    
    # Test different temperature values
    temperatures = [0.1, 0.5, 0.9]
    
    for temp in temperatures:
        print(f"\n🌡️ Temperature: {temp}")
        print("-" * 30)
        
        try:
            response = client.chat.completions.create(
                model=model,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=100,
                temperature=temp
            )
            
            print(f"Response: {response.choices[0].message.content[:150]}...")
            
        except Exception as e:
            print(f"Error with temperature {temp}: {e}")

test_different_parameters()

## சேவை ஆரோக்கிய சோதனை

விரிவான சேவை ஆரோக்கியம் மற்றும் திறன் சோதனை:


In [ ]:
def comprehensive_health_check():
    """Perform comprehensive health check of the service."""
    print("🏥 Comprehensive Health Check")
    print("=" * 50)
    
    # 1. Check model listing
    try:
        models_response = client.models.list()
        available_models = [m.id for m in models_response.data]
        print(f"✅ Model listing: SUCCESS")
        print(f"   📋 Available models: {available_models}")
        
        if model in available_models:
            print(f"   ✅ Current model '{model}' is available")
        else:
            print(f"   ⚠️ Current model '{model}' not found in available models")
    except Exception as e:
        print(f"❌ Model listing: FAILED - {e}")
    
    # 2. Test basic completion
    try:
        test_response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Say 'Health check successful'"}],
            max_tokens=10
        )
        print(f"✅ Basic completion: SUCCESS")
        print(f"   💬 Response: {test_response.choices[0].message.content}")
    except Exception as e:
        print(f"❌ Basic completion: FAILED - {e}")
    
    # 3. Test streaming
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": "Count to 3"}],
            max_tokens=20,
            stream=True
        )
        
        stream_content = ""
        chunk_count = 0
        for chunk in stream:
            if chunk.choices[0].delta.content:
                stream_content += chunk.choices[0].delta.content
                chunk_count += 1
        
        print(f"✅ Streaming: SUCCESS")
        print(f"   📦 Chunks received: {chunk_count}")
        print(f"   💬 Streamed content: {stream_content.strip()}")
    except Exception as e:
        print(f"❌ Streaming: FAILED - {e}")
    
    # 4. Provider-specific information
    print(f"\n📊 Configuration Summary:")
    print(f"   🏢 Provider: {provider}")
    print(f"   🤖 Model: {model}")
    if provider == "foundry":
        print(f"   🏠 Foundry SDK Available: {FOUNDRY_SDK_AVAILABLE}")
        print(f"   🔗 Base URL: {FOUNDRY_BASE_URL}")
    elif provider == "azure":
        print(f"   🌐 Azure Endpoint: {os.environ.get('AZURE_OPENAI_ENDPOINT', 'Not set')}")
        print(f"   🔑 API Version: {os.environ.get('AZURE_OPENAI_API_VERSION', 'Not set')}")

comprehensive_health_check()

## இடைநிலை சோதனை

இந்த செல் மூலம் உங்கள் சொந்த உத்தரவுகளை இடைநிலையாக சோதிக்கவும்:


In [ ]:
# Interactive testing - modify the prompt below
custom_prompt = "Explain the concept of 'edge AI' and why it's becoming important."
use_streaming = True  # Set to False for regular completion

print(f"👤 Custom Prompt: {custom_prompt}\n")

if use_streaming:
    custom_response = streaming_chat(custom_prompt, max_tokens=250)
else:
    custom_response = simple_chat(custom_prompt, max_tokens=250)
    print(f"🤖 Assistant: {custom_response}")

## சுருக்கம் மற்றும் அடுத்த படிகள்

இந்த நோட்புக் OpenAI SDK ஒருங்கிணைப்பின் மேம்பட்ட அம்சங்களை விளக்குகிறது:

### ✅ முக்கிய அம்சங்கள்

1. **பல வழங்குநர் ஆதரவு**: Azure OpenAI மற்றும் Foundry Local இடையே எளிதான மாறுதல்
2. **ஸ்ட்ரீமிங் பதில்கள்**: சிறந்த பயனர் அனுபவத்திற்காக நேரடி டோக்கன் உருவாக்கம்
3. **உரையாடல் மேலாண்மை**: பல முறை உரையாடல்களை சூழலுடன் நிர்வகித்தல்
4. **செயல்திறன் அளவீடு**: பதிலளிக்கும் நேரத்தை அளவிடுதல் மற்றும் பகுப்பாய்வு
5. **விரிவான ஆரோக்கிய சோதனைகள்**: சேவை சரிபார்ப்பு மற்றும் கண்டறிதல்
6. **பிழை கையாளுதல்**: வலுவான பிழை கையாளுதல் மற்றும் மாற்று வழிகள்

### 🏆 Foundry Local vs Azure OpenAI

| அம்சம் | Foundry Local | Azure OpenAI |
|--------|---------------|---------------|
| **தனியுரிமை** | ✅ தரவுகள் உள்ளூரில் இருக்கும் | ⚠️ தரவுகள் கிளவுடுக்கு அனுப்பப்படும் |
| **தாமதம்** | ✅ குறைவு (உள்ளூர் முடிவு) | ⚠️ அதிகம் (நெட்வொர்க் சார்பு) |
| **செலவு** | ✅ இலவசம் (ஹார்ட்வேருக்கு பிறகு) | 💰 டோக்கன் ஒன்றுக்கு செலவு |
| **ஆஃப்லைன்** | ✅ இணையம் இல்லாமல் செயல்படும் | ❌ இணையம் தேவை |
| **மாதிரி வகைகள்** | ⚠️ வரையறுக்கப்பட்ட தேர்வு | ✅ முழு மாதிரி அணுகல் |
| **மிகைப்படுத்தல்** | ⚠️ ஹார்ட்வேரை சார்ந்தது | ✅ வரையறையற்ற மிகைப்படுத்தல் |

### 🚀 அடுத்த படிகள்

- **Sample 04**: Chainlit உரையாடல் பயன்பாட்டை உருவாக்குதல்
- **Sample 05**: பல முகவர் ஒருங்கிணைப்பு அமைப்புகள்
- **Sample 06**: புத்திசாலி மாதிரி வழிமாற்று
- **தயாரிப்பு வெளியீடு**: மிகைப்படுத்தல் மற்றும் கண்காணிப்பு கருத்துக்கள்

### 💡 சிறந்த நடைமுறைகள்

1. **வழங்குநர்களுக்கு இடையே மாற்று வழிகளை எப்போதும் செயல்படுத்தவும்**
2. **நீண்ட பதில்களுக்கு ஸ்ட்ரீமிங்கை பயன்படுத்தவும்** உணரப்படும் செயல்திறனை மேம்படுத்த
3. **தயாரிப்பு பயன்பாடுகளுக்கு சரியான பிழை கையாளுதலை செயல்படுத்தவும்**
4. **பதிலளிக்கும் நேரம் மற்றும் செலவுகளை கண்காணிக்கவும்** வெவ்வேறு வழங்குநர்களுக்கு
5. **உங்கள் குறிப்பிட்ட தேவைகளின் அடிப்படையில் சரியான வழங்குநரை தேர்ந்தெடுக்கவும்**



---

**குறிப்பு**:  
இந்த ஆவணம் [Co-op Translator](https://github.com/Azure/co-op-translator) என்ற AI மொழிபெயர்ப்பு சேவையை பயன்படுத்தி மொழிபெயர்க்கப்பட்டுள்ளது. எங்கள் நோக்கம் துல்லியமாக இருக்க வேண்டும் என்பதுதான், ஆனால் தானியங்கி மொழிபெயர்ப்புகளில் பிழைகள் அல்லது துல்லியமின்மைகள் இருக்கக்கூடும் என்பதை தயவுசெய்து கவனத்தில் கொள்ளவும். அதன் தாய்மொழியில் உள்ள மூல ஆவணம் அதிகாரப்பூர்வ ஆதாரமாக கருதப்பட வேண்டும். முக்கியமான தகவல்களுக்கு, தொழில்முறை மனித மொழிபெயர்ப்பு பரிந்துரைக்கப்படுகிறது. இந்த மொழிபெயர்ப்பைப் பயன்படுத்துவதால் ஏற்படும் எந்த தவறான புரிதல்கள் அல்லது தவறான விளக்கங்களுக்கு நாங்கள் பொறுப்பல்ல.
